In [1]:
using BenchmarkTools
using ExtractRandom

In [2]:
types = [Vector{Bool}, BitVector, Vector{Int8}, Vector{Int64}]
data_sizes = 2.^(6:4:30)
a_sizes = [(512, 1024)]
const suite = BenchmarkGroup(["two_universal"])

for itype in types
    suite[itype] = BenchmarkGroup()
    for o in data_sizes
        suite[itype][div(o, 2)] = BenchmarkGroup()
    end
end

for itype in types
    for data_size in data_sizes
        for a_size in a_sizes
            (data_size % a_size[2] == 0 && data_size ≥ a_size[2]) || continue
            
            a = rand(eltype(itype), a_size)
            v = rand(eltype(itype), data_size)
            
            multiple = div(length(v), size(a, 2))
            if eltype(itype) <: Bool
                a = convert(BitMatrix, a)
                v = convert(BitArray, v)
                n = div(length(v), size(a, 2)) * size(a, 1)
                x = BitVector(n)
            else
                n = round(Int64, (div(length(v), size(a, 2)) * size(a, 1)) / 8sizeof(eltype(itype)), RoundUp)
                x = convert(itype, zeros(eltype(itype), n))
            end
            suite[itype][div(data_size, 2)][a_size] = @benchmarkable two_universal!($(a), $(v), $(x))
        end
    end
end


In [ ]:
tune!(suite)

In [ ]:
 results = run(suite, verbose = true, seconds = 1)

In [3]:
results[Vector{Int64}][512][(64,128)]

LoadError: UndefVarError: results not defined

In [4]:
showall(suite)

4-element BenchmarkTools.BenchmarkGroup:
  tags: ["two_universal"]
  Array{Bool,1} => 7-element BenchmarkTools.BenchmarkGroup:
	  tags: []
	  512 => 1-element BenchmarkTools.BenchmarkGroup:
		  tags: []
		  (512,1024) => Benchmark(evals=1, seconds=5.0, samples=10000)
	  2097152 => 1-element BenchmarkTools.BenchmarkGroup:
		  tags: []
		  (512,1024) => Benchmark(evals=1, seconds=5.0, samples=10000)
	  32 => 0-element BenchmarkTools.BenchmarkGroup:
		  tags: []
	  33554432 => 1-element BenchmarkTools.BenchmarkGroup:
		  tags: []
		  (512,1024) => Benchmark(evals=1, seconds=5.0, samples=10000)
	  131072 => 1-element BenchmarkTools.BenchmarkGroup:
		  tags: []
		  (512,1024) => Benchmark(evals=1, seconds=5.0, samples=10000)
	  8192 => 1-element BenchmarkTools.BenchmarkGroup:
		  tags: []
		  (512,1024) => Benchmark(evals=1, seconds=5.0, samples=10000)
	  536870912 => 1-element BenchmarkTools.BenchmarkGroup:
		  tags: []
		  (512,1024) => Benchmark(evals=1, seconds=5.0, samples=10000)
  Arr

In [13]:
run(suite[Vector{Int64}][8192][(512, 1024)], samples=3, seconds=60)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     11.912 s (0.00% GC)
  median time:      12.205 s (0.00% GC)
  mean time:        12.269 s (0.00% GC)
  maximum time:     12.690 s (0.00% GC)
  --------------
  samples:          3
  evals/sample:     1

In [15]:
run(suite[Vector{Int8}][8192][(512, 1024)], samples=20, seconds=60)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.356 s (0.00% GC)
  median time:      4.425 s (0.00% GC)
  mean time:        4.432 s (0.00% GC)
  maximum time:     4.554 s (0.00% GC)
  --------------
  samples:          14
  evals/sample:     1